In [1]:
# Database connection and CSV parsing 
import pymysql

In [2]:
db_name = "majorproject"
db_host = "localhost"
db_username = "root"
db_password = ""

In [3]:
try:
    dbConn = pymysql.connect(host = db_host, port = int(3306), user = db_username, password = db_password, db = db_name)
except e:
    print (e)

In [4]:
if dbConn:
    print ("Successfully conntected.")
else:
        print ("Unable to connect the DB.")

Successfully conntected.


In [5]:
# CSV file read.
fileName = "report.csv"
csvData = open(fileName, 'r')
allContents = csvData.readlines()


In [1]:
# print(allContents[0])

In [7]:
dataContents = allContents[1:]
print(len(dataContents))

39592


In [8]:
dataContents[0]

'ff72f0faf0574419-8345aebd86a848f8,3529,Sun Apr 12 13:01:16 NPT 2020,54662e39-6f8abdcb,1,130,303fa913-8989e247,2,125,6a80d33e-83859f24,1,80,871c68b0-3c4bc1b0,2,820,226e0f5e-8b946a36,2,160,419f9d0e-a8fa881e,2,110,2fae974c-c6fb6f9d,1,164,1c8efa6a-db242122,1,325,2b8ddc68-5e107d12,2,180,7ad41de2-bb1053b7,2,20,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n'

In [9]:
def divide_chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
def build_key(fields):
    return '-'.join(fields)

In [11]:
# collects all the bill items in the form of dictionary.
orderBills = []
a = 0
for entry in dataContents:
    entries = entry.split(",")
    firstPart = [item.strip() for item in entries[0:3]]
    firstPart.pop(1) # Removes total quantity.    
    secondPart = entries[3:]
    secondPart = [i for i in secondPart if len(i.strip()) > 0]
    sliced = list(divide_chunks(secondPart, 3))
    
    sliced = [bill_item for bill_item in sliced if len(bill_item) > 0]
    for orderQty in sliced:
        if len(orderQty) == 3:
            rowData = firstPart + orderQty
            orderBills.append(rowData)
        

In [12]:
len(orderBills)

292160

In [13]:
def hasEmpty(items):
    return items[0]=="" or items[1]=="" or items[2]=="" or items[3]=="" or items[4]==""

In [14]:
orderBills = [item for item in orderBills if not hasEmpty(item)]

count = 0
for i in orderBills:
    if(len(i) == 5):
        count = count + 1

print(count)

291771


In [15]:
orderBills

[['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '54662e39-6f8abdcb',
  '1',
  '130'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '303fa913-8989e247',
  '2',
  '125'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '6a80d33e-83859f24',
  '1',
  '80'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '871c68b0-3c4bc1b0',
  '2',
  '820'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '226e0f5e-8b946a36',
  '2',
  '160'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '419f9d0e-a8fa881e',
  '2',
  '110'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '2fae974c-c6fb6f9d',
  '1',
  '164'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '1c8efa6a-db242122',
  '1',
  '325'],
 ['ff72f0faf0574419-8345aebd86a848f8',
  'Sun Apr 12 13:01:16 NPT 2020',
  '2b8ddc68-5e107d12',
 

In [16]:
# check if the table is already created, if so, make ready with empty data.
def checkTableExists(dbConnection, tablename):
    result = False
    dbcur = dbConnection.cursor()
    dbcur.execute("""SELECT COUNT(*) FROM information_schema.tables WHERE table_name = '{0}' """
                  .format(tablename.replace('\'', '\'\'')))
    if dbcur.fetchone()[0] == 1:
        result = True

    dbcur.close()
    return result


In [17]:
# check table name exists and make it empty before uploading all the data.
checkTable = checkTableExists(dbConn, "bills")
dbCursor = dbConn.cursor()
if checkTable:
    dbCursor.execute("DELETE FROM `bills`")
    dbConn.commit()

In [18]:
from datetime import datetime
from dateutil import parser, tz

# Give date time format.
dtFormat = "%a %b %d %H:%M:%S %Z %Y"
custom_tz = tz.gettz("Asia/Kathmandu")  # Define the custom timezone

# Perform insert operation for all the data.
for row in orderBills:
    query = ''

    # Data casting is necessary for the field type in the Database table.
    parsedDate = parser.parse(str(row[1]), tzinfos={"NPT": custom_tz})
    row[1] = parsedDate.strftime('%Y-%m-%d %H:%M:%S')
    # Build query from the given row data.
    # INSERT INTO `bills` (`customer_id`, `order_date`, `product_id`, `quantity`, `price`) VALUES ('ff72f0faf0574419-8345aebd86a848f8', '2020-04-12 13:01:16', '54662e39-6f8abdcb', '1', '130');
    query = "INSERT INTO `bills` (`customer_id`, `order_date`, `product_id`, `quantity`, `price`) VALUES %r;" % (tuple(row),)
    dbCursor.execute(query)
        

dbConn.commit()
print("All inserted successfully.")

All inserted successfully.
